In [141]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os

In [142]:
train_pneumonia_dir=os.path.join('/content/drive/My Drive/chest_xray/train/PNEUMONIA')
train_normal_dir=os.path.join('/content/drive/My Drive/chest_xray/train/NORMAL')

In [143]:
val_pneumonia_dir=os.path.join('/content/drive/My Drive/chest_xray/val/PNEUMONIA')
val_normal_dir=os.path.join('/content/drive/My Drive/chest_xray/val/NORMAL')

In [144]:
test_pneumonia_dir=os.path.join('/content/drive/My Drive/chest_xray/test/PNEUMONIA')
test_normal_dir=os.path.join('/content/drive/My Drive/chest_xray/test/NORMAL')

In [145]:
X_train_pnuemonia=print(f'length of train pneumonia:{len(os.listdir(train_pneumonia_dir))}')
X_train_normal=print(f'length of train normal:{len(os.listdir(train_normal_dir))}')

length of train pneumonia:3875
length of train normal:1345


In [146]:
print(f'length of val pneumonia:{len(os.listdir(val_pneumonia_dir))}')
print(f'length of val normal:{len(os.listdir(val_normal_dir))}')

length of val pneumonia:8
length of val normal:8


In [147]:
print(f'length of test pneumonia:{len(os.listdir(test_pneumonia_dir))}')
print(f'length of test normal:{len(os.listdir(test_normal_dir))}')

length of test pneumonia:390
length of test normal:234


In [148]:
train_gen=ImageDataGenerator(rescale=1/255)
test_gen=ImageDataGenerator(rescale=1/255)
val_gen=ImageDataGenerator(rescale=1/255)

In [149]:
tar_size = (512,256)

In [150]:
train_generator=train_gen.flow_from_directory('/content/drive/My Drive/chest_xray/train',
                                              target_size=tar_size,batch_size=32,class_mode='binary')

Found 5220 images belonging to 2 classes.


In [151]:
test_generator=test_gen.flow_from_directory('/content/drive/My Drive/chest_xray/test',target_size=tar_size,
                                            batch_size=32,class_mode='binary')

Found 624 images belonging to 2 classes.


In [152]:
val_generator=val_gen.flow_from_directory('/content/drive/My Drive/chest_xray/val',target_size=tar_size,
                                          batch_size=32,class_mode='binary')

Found 16 images belonging to 2 classes.


In [153]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [154]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(512, 256, 3),padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2,padding='same'),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(2,2,padding='same'),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(2,2,padding='same'),
    
    # Flatten the results to feed into a DNN
    #tf.keras.layers.Flatten(input_shape=(500,300,1)),
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [155]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 512, 256, 16)      448       
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 256, 128, 16)      0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 256, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_44 (MaxPooling (None, 128, 64, 32)       0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 128, 64, 64)       18496     
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 64, 32, 64)        0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 131072)          

In [156]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [157]:
from sklearn.utils import class_weight
import numpy as np

my_class_weights = class_weight.compute_class_weight(
                                          'balanced',
                                          np.unique(train_generator.classes),
                                          train_generator.classes)

weight = {i : my_class_weights[i] for i in range(len(my_class_weights))}

In [158]:
weight

{0: 1.9405204460966543, 1: 0.6735483870967742}

In [159]:
history = model.fit(
      train_generator,
      epochs=15,
      verbose=1,
      validation_data=val_generator,
      class_weight = weight,callbacks=[callbacks])

Epoch 1/15
164/164 [==============================] - 91s 553ms/step - loss: 0.3159 - accuracy: 0.8977 - val_loss: 0.1154 - val_accuracy: 0.9375
Epoch 2/15
164/164 [==============================] - ETA: 0s - loss: 0.0918 - accuracy: 0.9623
Reached 95% accuracy so cancelling training!
164/164 [==============================] - 62s 378ms/step - loss: 0.0918 - accuracy: 0.9623 - val_loss: 0.0362 - val_accuracy: 1.0000
